In [8]:
# Instalar dependências
# Instalar Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get install -y -f
!rm google-chrome-stable_current_amd64.deb

# Remover instalação de chromium-chromedriver (não precisamos mais)
# !apt-get update > /dev/null
# !apt install chromium-chromedriver > /dev/null
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Instalar selenium, python-dotenv e webdriver_manager
!pip install selenium python-dotenv webdriver_manager > /dev/null

# Código ajustado
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Importar ChromeDriverManager do webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from dotenv import load_dotenv
import os
import logging
import time
import traceback

# Configurar log
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# XPATHs utilizados
XPATHS = {
    "usuario": "//input[@placeholder='Usuário' and @type='text']",
    "senha": "//input[@placeholder='Senha' and @type='password']",
    "botao_login": "//button[@type='submit' and contains(text(), 'Entrar')]",
    "codigo_icare": "//input[@id='id' and @type='text']",
    "menu_financeiro": "//a[text()='Financeiro']",
    "menu_faturas": "//a[text()='Faturas']",
    "radio_faturas": "//input[@type='radio' and @value='todos']",
    "expandir_fatura": "//input[@type='image' and contains(@src, 'mais.png')]",
    "tabela_faturas": "//div[@id='detailInvoice1']//tbody",
    "linhas_faturas": "//div[@id='detailInvoice1']//tbody/tr"
}

def realizar_login(driver, usuario, senha):
    logging.info("Realizando login...")
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, XPATHS["usuario"]))).send_keys(usuario)
    wait.until(EC.presence_of_element_located((By.XPATH, XPATHS["senha"]))).send_keys(senha)
    wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["botao_login"]))).click()

def inserir_codigo_icare(driver, codigo):
    logging.info("Inserindo código ICare...")
    wait = WebDriverWait(driver, 10)
    campo_icare = wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["codigo_icare"])))
    campo_icare.send_keys(codigo)
    campo_icare.send_keys(Keys.ENTER)
    time.sleep(3)

def navegar_para_faturas(driver):
    logging.info("Navegando até faturas...")
    wait = WebDriverWait(driver, 10)
    wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["menu_financeiro"]))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["menu_faturas"]))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["radio_faturas"]))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, XPATHS["expandir_fatura"]))).click()

def processar_faturas(driver):
    logging.info("Processando faturas...")
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, XPATHS["tabela_faturas"])))
    linhas = driver.find_elements(By.XPATH, XPATHS["linhas_faturas"])
    if not linhas:
        logging.warning("Nenhuma fatura encontrada.")
        return
    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME, "td")
        valores = [col.text.strip() for col in colunas]
        print(valores)

def main():
    # Configurações do Chrome para Colab
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Usar ChromeDriverManager para obter o caminho do chromedriver compatível
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Carregar variáveis de ambiente
    load_dotenv()
    usuario = os.getenv("USUARIO_ICARE", "seu_usuario")
    senha = os.getenv("SENHA_ICARE", "sua_senha")

    try:
        driver.get("https://icareclientes-av.sky.com.br/")
        realizar_login(driver, usuario, senha)

        # Espera e troca de janela
        # Aumentar o tempo de espera para garantir que a nova janela apareça
        WebDriverWait(driver, 20).until(lambda d: len(d.window_handles) > 1)
        login_window = driver.current_window_handle
        for window in driver.window_handles:
            if window != login_window:
                driver.switch_to.window(window)
                break

        logging.info("Janela ICare Clientes ativa.")
        inserir_codigo_icare(driver, "1530404719")
        navegar_para_faturas(driver)
        processar_faturas(driver)

    except Exception as e:
        logging.error(f"Erro no processo: {e}")
        logging.error(traceback.format_exc())
    finally:
        if 'driver' in locals() and driver is not None:
             driver.quit()


# Executar
main()


--2025-06-12 17:14:12--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.2.91, 142.251.2.136, 142.251.2.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.2.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117322064 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 111.89M  22.8MB/s    in 5.8s    

2025-06-12 17:14:19 (19.5 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [117322064/117322064]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126568 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (137.0.7151.103-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libv

ERROR:root:Erro no processo: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=137.0.7151.103)
Stacktrace:
#0 0x55ab02a7bc9a <unknown>
#1 0x55ab025216e0 <unknown>
#2 0x55ab025187e7 <unknown>
#3 0x55ab02509699 <unknown>
#4 0x55ab0250b3e0 <unknown>
#5 0x55ab02509a3d <unknown>
#6 0x55ab025093af <unknown>
#7 0x55ab02509085 <unknown>
#8 0x55ab02506eee <unknown>
#9 0x55ab025076ca <unknown>
#10 0x55ab02524c09 <unknown>
#11 0x55ab025bfed5 <unknown>
#12 0x55ab02598e32 <unknown>
#13 0x55ab025bf2cc <unknown>
#14 0x55ab02598c03 <unknown>
#15 0x55ab0256547b <unknown>
#16 0x55ab025660e1 <unknown>
#17 0x55ab02a4044b <unknown>
#18 0x55ab02a4437f <unknown>
#19 0x55ab02a27f89 <unknown>
#20 0x55ab02a44f18 <unknown>
#21 0x55ab02a0c6df <unknown>
#22 0x55ab02a69308 <unknown>
#23 0x55ab02a694e6 <unknown>
#24 0x55ab02a7ab76 <unknown>
#25 0x798e7f7acac3 <unknown>

ERROR:root:Traceback (most recent call last):
  File "<ipython-input-8-1300537576>", line 102, in main
    driver.get("http

In [5]:
!pip install selenium
!pip install python-dotenv


In [3]:
!apt-get update > /dev/null
!apt install chromium-chromedriver > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
